In [1]:
"""
Processing the data
"""
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler


def process_data(train, test, lags):
    """Process data
    Reshape and split train\test data.

    # Arguments
        train: String, name of .csv train file.
        test: String, name of .csv test file.
        lags: integer, time lag.
    # Returns
        X_train: ndarray.
        y_train: ndarray.
        X_test: ndarray.
        y_test: ndarray.
        scaler: StandardScaler.
    """
    attr = 'VFlow'
    df1 = pd.read_csv(train, encoding='utf-8').fillna(0)
    df2 = pd.read_csv(test, encoding='utf-8').fillna(0)

    # scaler = StandardScaler().fit(df1[attr].values)
    scaler = MinMaxScaler(feature_range=(0, 1)).fit(df1[attr].values.reshape(-1, 1))
    flow1 = scaler.transform(df1[attr].values.reshape(-1, 1)).reshape(1, -1)[0]
    flow2 = scaler.transform(df2[attr].values.reshape(-1, 1)).reshape(1, -1)[0]

    train, test = [], []
    for i in range(lags, len(flow1)):
        train.append(flow1[i - lags: i + 1])
    for i in range(lags, len(flow2)):
        test.append(flow2[i - lags: i + 1])

    train = np.array(train)
    test = np.array(test)
    np.random.shuffle(train)

    X_train = train[:, :-1]
    y_train = train[:, -1]
    X_test = test[:, :-1]
    y_test = test[:, -1]

    return X_train, y_train, X_test, y_test, scaler

In [2]:
def train_model(model, X_train, y_train, name, scats_number, config):
    """train
    train a single model.

    # Arguments
        model: Model, NN model to train.
        X_train: ndarray(number, lags), Input data for train.
        y_train: ndarray(number, ), result data for train.
        name: String, name of model.
        config: Dict, parameter for train.
    """

    model.compile(loss="mse", optimizer="rmsprop", metrics=['mape'])
    # early = EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='auto')
    hist = model.fit(
        X_train, y_train,
        batch_size=config["batch"],
        epochs=config["epochs"],
        validation_split=0.05,
        verbose=0)

    #Create folder structure: model/<model_name>/<scats_number>
    model_folder = f'model/{name}/{scats_number}'
    os.makedirs(model_folder, exist_ok=True)

    #Save the model and loss history
    model.save(f'{model_folder}/{name}_{scats_number}.h5')
    df = pd.DataFrame.from_dict(hist.history)
    df.to_csv(f'{model_folder}/{name}_{scats_number}_loss.csv', encoding='utf-8', index=False)

In [3]:
import os

def do_train(model_name):
    lag = 12
    config = {"batch": 256, "epochs": 500}

    train_folder = 'intersection/train/'
    test_folder = 'intersection/test/'

    for filename in os.listdir(train_folder):
        if filename.endswith(".csv"):
            scats_number = filename.split('_')[1].split('.')[0]
            train_file = os.path.join(train_folder, filename)
            test_file = os.path.join(test_folder, f"test_{scats_number}.csv")

            print(f"Processing SCATS Number: {scats_number}")

            X_train, y_train, X_test, y_test, scaler = process_data(train_file, test_file, lag)

            model_file_name = model_name

            if model_name == 'lstm':
                X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
                m = get_lstm([12, 64, 64, 1])
                train_model(m, X_train, y_train, model_file_name, scats_number, config)

            elif model_name == 'gru':
                X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
                m = get_gru([12, 64, 64, 1])
                train_model(m, X_train, y_train, model_file_name, scats_number, config)

            elif model_name == 'saes':
                X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
                m = get_saes([12, 400, 400, 400, 1])
                train_seas(m, X_train, y_train, model_file_name, scats_number, config)
            
            elif model_name == 'nt_saes':
                X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
                m = get_nt_saes([12, 400, 400, 400, 1])
                train_nt_saes(m, X_train, y_train, model_file_name, scats_number, config)

            print(f"Saved model for SCATS Number: {scats_number}")


In [4]:
"""
Defination of NN model
"""
from keras.layers import Dense, Dropout, Activation
from keras.layers import Input, LSTM, GRU, Dense 
from keras.models import Sequential


def get_lstm(units):
    """LSTM(Long Short-Term Memory)
    Build LSTM Model.

    # Arguments
        units: List(int), number of input, output and hidden units.
    # Returns
        model: Model, nn model.
    """

    model = Sequential()
    model.add(Input(shape=(units[0], 1)))
    model.add(LSTM(units[1], return_sequences=True))
    model.add(LSTM(units[2]))
    model.add(Dropout(0.2))
    model.add(Dense(units[3], activation='sigmoid'))

    return model


def get_gru(units):
    """GRU(Gated Recurrent Unit)
    Build GRU Model.

    # Arguments
        units: List(int), number of input, output and hidden units.
    # Returns
        model: Model, nn model.
    """

    model = Sequential()
    model.add(GRU(units[1], input_shape=(units[0], 1), return_sequences=True))
    model.add(GRU(units[2]))
    model.add(Dropout(0.2))
    model.add(Dense(units[3], activation='sigmoid'))

    return model


def _get_nt_sae(inputs, hidden, output):
    """SAE(Auto-Encoders)
    Build SAE Model.

    # Arguments
        inputs: Integer, number of input units.
        hidden: Integer, number of hidden units.
        output: Integer, number of output units.
    # Returns
        model: Model, nn model.
    """

    model = Sequential()
    model.add(Dense(hidden, input_dim=inputs, name='hidden'))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(output, activation='sigmoid'))

    return model


def get_nt_saes(layers):
    """SAEs(Stacked Auto-Encoders)
    Build SAEs Model.

    # Arguments
        layers: List(int), number of input, output and hidden units.
    # Returns
        models: List(Model), List of SAE and SAEs.
    """
    sae1 = _get_nt_sae(layers[0], layers[1], layers[-1])
    sae2 = _get_nt_sae(layers[1], layers[2], layers[-1])
    sae3 = _get_nt_sae(layers[2], layers[3], layers[-1])

    saes = Sequential()
    saes.add(Dense(layers[1], input_dim=layers[0], name='hidden1'))
    saes.add(Activation('sigmoid'))
    saes.add(Dense(layers[2], name='hidden2'))
    saes.add(Activation('sigmoid'))
    saes.add(Dense(layers[3], name='hidden3'))
    saes.add(Activation('sigmoid'))
    saes.add(Dropout(0.2))
    saes.add(Dense(layers[4], activation='sigmoid'))

    models = [sae1, sae2, sae3, saes]

    return models

2024-10-16 22:04:23.321113: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 22:04:23.389584: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 22:04:23.408128: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 22:04:23.525316: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-16 22:04:24.442760: W tensorflow/compiler/tf2

In [5]:
def train_nt_saes(models, X_train, y_train, name, scats_number, config):
    """train
    train the SAEs model.

    # Arguments
        models: List, list of SAE model.
        X_train: ndarray(number, lags), Input data for train.
        y_train: ndarray(number, ), result data for train.
        name: String, name of model.
        config: Dict, parameter for train.
    """

    temp = X_train
    # early = EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='auto')

    for i in range(len(models) - 1):
        if i > 0:
            p = models[i - 1]
            hidden_layer_model = Model(inputs=p.inputs,
                                       outputs=p.get_layer('hidden').output)
            temp = hidden_layer_model.predict(temp)

        m = models[i]
        m.compile(loss="mse", optimizer="rmsprop", metrics=['mape'])

        m.fit(temp, y_train, batch_size=config["batch"],
              epochs=config["epochs"],
              validation_split=0.05)

        models[i] = m

    saes = models[-1]
    for i in range(len(models) - 1):
        weights = models[i].get_layer('hidden').get_weights()
        saes.get_layer('hidden%d' % (i + 1)).set_weights(weights)

    train_model(saes, X_train, y_train, name, scats_number, config)

## warning: this stuff will take ages to run and needs GPU time

In [6]:
from keras.models import Model
from keras.callbacks import EarlyStopping

In [ ]:
do_train('sae')

In [ ]:
do_train('nt_saes')

START NEXT TRAIN FROM HERE!!!!!!!!!!!!!!!!

In [ ]:
do_train('gru')

In [ ]:
do_train('lstm')

## end of warning

In [7]:
import math
import warnings
import numpy as np
import pandas as pd
from keras.models import load_model
from tensorflow.keras.utils import plot_model
import sklearn.metrics as metrics
import matplotlib as mpl
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

from keras.src.legacy.saving import legacy_h5_format

In [30]:
def MAPE(y_true, y_pred):
    """Mean Absolute Percentage Error
    Calculate the mape.

    # Arguments
        y_true: List/ndarray, ture data.
        y_pred: List/ndarray, predicted data.
    # Returns
        mape: Double, result data for train.
    """

    y = [x for x in y_true if x > 0]
    y_pred = [y_pred[i] for i in range(len(y_true)) if y_true[i] > 0]

    num = len(y_pred)
    sums = 0

    for i in range(num):
        tmp = abs(y[i] - y_pred[i]) / y[i]
        sums += tmp

    mape = sums * (100 / num)

    return mape


def eva_regress(y_true, y_pred, silent=False):
    """Evaluation
    evaluate the predicted resul.

    # Arguments
        y_true: List/ndarray, ture data.
        y_pred: List/ndarray, predicted data.
    """

    mape = MAPE(y_true, y_pred)
    vs = metrics.explained_variance_score(y_true, y_pred)
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)
    
    if not silent:
        print('explained_variance_score:%f' % vs)
        print('mape:%f%%' % mape)
        print('mae:%f' % mae)
        print('mse:%f' % mse)
        print('rmse:%f' % math.sqrt(mse))
        print('r2:%f' % r2)
        
    return mape, vs, mae, mse, r2

def plot_results(y_true, y_preds, names):
    """Plot
    Plot the true data and predicted data.

    # Arguments
        y_true: List/ndarray, ture data.
        y_pred: List/ndarray, predicted data.
        names: List, Method names.
    """
    d = '2016-3-4 00:00'
    x = pd.date_range(d, periods=96, freq='15min')

    fig = plt.figure()
    ax = fig.add_subplot(111)

    ax.plot(x, y_true, label='True Data')
    for name, y_pred in zip(names, y_preds):
        ax.plot(x, y_pred, label=name)

    plt.legend()
    plt.grid(True)
    plt.xlabel('Time of Day')
    plt.ylabel('Flow')

    date_format = mpl.dates.DateFormatter("%H:%M")
    ax.xaxis.set_major_formatter(date_format)
    fig.autofmt_xdate()


def main():
    lag = 12
    train_folder = 'intersection/train/'
    test_folder = 'intersection/test/'

    models = ['lstm', 'gru', 'sae', 'nt_saes']
    names = ['LSTM', 'GRU', 'SAE', 'NT_SAEs']

    stats_row_list = []
    
    for filename in os.listdir(train_folder):
        if filename.endswith(".csv"):
            scats_number = filename.split('_')[1].split('.')[0]
            train_file = os.path.join(train_folder, filename)
            test_file = os.path.join(test_folder, f"test_{scats_number}.csv")

            print(f"Evaluating for SCATS Number: {scats_number}")
            _, _, X_test, y_test, scaler = process_data(train_file, test_file, lag)
            y_test = scaler.inverse_transform(y_test.reshape(-1, 1)).reshape(1, -1)[0]

            y_preds = []

            models_to_remove = []
            for model_variant in models:
                
                try:
                    model = load_model(f"model/{model_variant}/{scats_number}/{model_variant}_{scats_number}.h5", custom_objects={'mse': 'mse'})
                except:
                    print(f'No model found for {model_variant} in {scats_number}... assuming not trained, removing from models to test.')
                    # don't remove during iteration as it will result in unintended behaviour
                    models_to_remove.append(model_variant)
                    continue
                
                X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
                predicted = model.predict(X_test)
                predicted = scaler.inverse_transform(predicted.reshape(-1, 1)).reshape(1, -1)[0]
                y_preds.append(predicted[:96])
    
                print(f"Evaluating {model_variant} for SCATS Number: {scats_number}")
                mape, vs, mae, mse, r2 = eva_regress(y_test, predicted, silent=True)
    
                model_stats = {
                    'scats': scats_number,
                    'model': model_variant,
                    'mape': mape,
                    'vs': vs,
                    'mae': mae,
                    'mse': mse,
                    'r2': r2
                }
    
                stats_row_list.append(model_stats)

            
            for model_variant in models_to_remove:
                models.remove(model_variant)

    stats_df = pd.DataFrame(stats_row_list)
    stats_df.set_index('scats')

    stats_df.to_csv('intersections_test_results.csv')
    #plot_results(y_test[:96], y_preds, names)

In [ ]:
main()